In [1]:
%pip install fastapi pydantic toml

Note: you may need to restart the kernel to use updated packages.


In [2]:
# from rs import skill_check

# def test_skill_check(metadata):
#     print(skill_check("sanity", None, metadata))

# metadata = {
#     "character": {"sanity":30},
#     "symptom":[
#         "incoherent speech",
#         "uncontrollable twitching, trembling",
#         "delusions of persecution",
#         "strange appetites (dirt, clay, etc...)",
#         "scratches, punches, and bruises all over the body"
#     ]
# }

# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)
# test_skill_check(metadata)

# print(metadata["character"])


In [3]:
import re
import tomllib
import functools
from typing import Literal

import ipywidgets as widgets
from IPython.display import display

from dotenv import load_dotenv

load_dotenv()

from rs import Message, Config, chat, skill_check
import d20

action_pattern = re.compile(r"(\d+). *(.+)")
skill_pattern = re.compile(r"[\w_]+")
skill_difficulty_pattern = re.compile(r"(easy|normal|hard)")

def next_round(b):
    global current_round, initial_messages
    current_round += 1
    initial_messages = config.initial_messages(current_round, config.metadata)

    print(initial_messages[-1].content)
    display(get_controls(initial_messages[-1]))

def get_user_message_tail():
    count = 0
    for msg in initial_messages:
        if msg.role == "user":
            count += 1

    is_final =  ", FINAL ROUND" if  current_round == len(config.rounds) else ""
    
    # print(config.metadata["character"])
    
    if "tags" in config.metadata["character"] and "insane" in config.metadata["character"]["tags"]:
        return f"TURN: {count + 1} {is_final} (I'm insane!)"
    
    return f"TURN: {count + 1} {is_final}"
    

def do_chat(role:Literal["user", "system", "assistant"], content: str):
    print(f"{content}\n\n")

    initial_messages.append(Message(role=role, content=content))
    msg = chat(initial_messages, lambda delta: print(delta, end=""))

    initial_messages.append(msg)
    display(get_controls(msg))


def on_skill(b, skill:str, difficulty:str):
    # print(config.metadata["character"])
    content = skill_check(skill, difficulty, config.metadata)
    content += f"\n{get_user_message_tail()}"
    do_chat("user", content)

def on_action(b, index:int):
    content = f"I select option {index}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_input(b, input:widgets.Text):
    content = f"{input.value}. {get_user_message_tail()}"
    do_chat("user", content)

def on_custom_action(b):
    hbox = widgets.HBox(layout=widgets.Layout(width="100%"))
    action_text = widgets.Text(layout=widgets.Layout(width="100%"))
    submit = widgets.Button(description="SUBMIT", layout=widgets.Layout(width="fit-content"))
    submit.on_click(functools.partial(on_custom_input, input=action_text))
    hbox.children += (action_text, submit) 
    display(hbox)

def get_controls(msg:Message) -> widgets.Box:
    dict = msg.dict()
    buttons = []
    if "possible actions" in dict.keys():
        matches = action_pattern.finditer(dict["possible actions"])
        for match in matches:
            btn = widgets.Button(description=match.group(1), tooltip=match.group(2), layout=widgets.Layout(width="auto"))
            btn.on_click(functools.partial(on_action, index=int(match.group(1))))
            buttons.append(btn)
    elif "skill" in dict.keys() and "difficulty" in dict.keys():
        match = skill_pattern.match(dict["skill"])
        skill = match.group(0)
        match = skill_difficulty_pattern.match(dict["difficulty"])
        difficulty = match.group(0)
        skill_button = widgets.Button(description=f"{skill.upper()} [{difficulty.upper()}]", layout=widgets.Layout(width="auto"))
        skill_button.on_click(functools.partial(on_skill, skill=skill, difficulty=difficulty))
        buttons.append(skill_button)
    elif "summary" in dict.keys():
        # add summary as a memory to the metadata
        # it will be used in the next round.
        # config.metadata["memories"] = []
        config.metadata["memories"].append(dict["summary"])
        next_btn = widgets.Button(description="NEXT ROUND", layout=widgets.Layout(width="auto"))
        next_btn.on_click(next_round)
        buttons.append(next_btn)
    
    custom_action_button = widgets.Button(description="CUSTOM")
    custom_action_button.on_click(on_custom_action)
    buttons.append(custom_action_button)

    return widgets.HBox(buttons)

# open config file
config_name = "data/ai_gf.toml"
config: Config | None = None
with open(config_name, "rb") as f:
    data = tomllib.load(f)
    config = Config(**data)

current_round = 0
initial_messages = config.initial_messages(current_round, config.metadata)

print(initial_messages[-1].content)
display(get_controls(initial_messages[-1]))

The year is 2023, you travelled to some distant mountains and forest in Hokkaido to do some scenic photography in the middle of the week during autumn. You wanted to get away from the city life of Tokyo just so you can make some time for yourself and think about the future and want to do in your life. As you are hiking through the quiet mountain pass, you hear a cry of a girl just in the distance. You quickly made your way towards the sound and found a beautiful young lady on the side of the road, and appears to have accidently injured herself.

What is your response?

possible actions:
1. Offer your assitance and then introduce yourself casually
2. Say a funny pick up line
3. Ignore her and keep walking


I select option 1. TURN: 1 


You approach the young lady with a concerned expression and offer your assistance. After helping her up, you introduce yourself, "Hi, I'm Hikaru Mikami. Are you okay? That looked like quite a fall."

Mitsuha looks up at you with gratitude in her big brown eyes. "Thank you so much! I'm Mitsuha Tomoe. I guess I wasn't watching where I was going. I'm a bit clumsy sometimes," she says with a cheeky smile. 

You both share a light moment, and it seems like she appreciates your help and the casual introduction. 

What would you like to do next?

possible actions:
1. Ask her if she needs any medical assistance or help getting back to where she's staying.
2. Suggest going to the nearest café or place where she can sit and rest.
3. Offer to accompany her for the rest of her hike for safety.

I select option 3. TURN: 2 


Mitsuha seems pleasantly surprised by your offer and nods enthusiastically. "That would be really nice, thank you! I was actually feeling a bit uneasy hiking alone after the fall," she admits, her quirky personality shining through as she adjusts her stylish modern mountain gear.

As you both continue the hike, the conversation flows easily between you two. Mitsuha shares her passion for art and her recent job with an art curator in Tokyo, while you find yourself opening up about your stressful banking job and your longing for a more fulfilling career path.

The hike becomes more enjoyable with Mitsuha's company, and you both seem to connect over shared interests and life aspirations.

What would you like to do next?

possible actions:
1. Share your dream of becoming a concept artist and how you ended up as a banker instead.
2. Ask Mitsuha more about her work in the art industry and her future aspirations.
3. Suggest taking a break and enjoying the scenic 

I select option 3. TURN: 3 


You suggest taking a break to enjoy the scenic view, and Mitsuha agrees with excitement. You both find a comfortable spot with a breathtaking view of the autumn colors blanketing the mountain range. The serene atmosphere and the beauty of nature around you provide a perfect backdrop for a moment of relaxation and connection.

As you both sit down, Mitsuha takes out a small sketchbook from her bag and begins to draw the landscape. She seems completely absorbed in her art, her expression one of pure focus and contentment. You watch her for a moment, admiring her passion and talent.

After a while, she notices your interest and offers to show you her sketch. "It's not much, but I love capturing moments like these," she says with a modest smile.

What would you like to do next?

possible actions:
1. Compliment her on her sketch and share your own aspirations in art.
2. Suggest taking a photo of the view together as a memory of this hike.
3. Ask if she would li

I select option 1. TURN: 4 


You compliment Mitsuha on her sketch, expressing genuine admiration for her talent. "This is really beautiful, Mitsuha. You've captured the view perfectly," you say, then hesitate for a moment before sharing your own aspirations. "You know, I've always dreamed of becoming a concept artist. I ended up in banking, but deep down, art is where my heart is."

Mitsuha looks at you with a newfound interest and nods understandingly. "That's really amazing, Hikaru. It's never too late to pursue what truly makes you happy," she encourages, her eyes reflecting genuine support. "Art has this incredible power to change lives, don't you think?"

Your conversation deepens as you both share your dreams and the obstacles you've faced. It feels refreshing to talk so openly with someone who understands and shares similar passions.

As the sun begins to set, casting a golden hue over the landscape, you realize how much you've enjoyed Mitsuha's company.

What would you like to

...

I select option 3. TURN: 1 


You need to make a skill check to crack a joke.
skill: Fast Talk
difficulty: normal

I am making a skill check using FAST against a difficulty of NORMAL.
And I rolled a 57 for a result of FAILURE.
TURN: 2 


With a chuckle, you attempt to make a light-hearted joke about the sake, saying, "Maybe we should ask if the sake is strong enough to paint our evening with unforgettable colors, or if it'll just end up blurring the lines of our sketches." However, your delivery doesn't land as smoothly as you hoped, and there's a brief awkward silence.

Mitsuha, trying to keep the mood light, smiles politely and says, "Well, I guess there's only one way to find out, right? Let's give it a shot, but maybe just a small one for me. I'm not much of a drinker, and I wouldn't want my sketching hand to start drawing circles by itself."

The owner, picking up on the attempt to keep things jovial, nods and brings over a small bottle of sake, pouring a modest amount for both of you.

As you both sip the sake, Mitsuha's cheeks begin to flush slightly from the alcohol. She seems more relaxed 

I select option 2. TURN: 3 


Curious about her work, you steer the conversation towards her job at the art gallery. "So, working in an art gallery must be fascinating. You get to be around all these incredible pieces every day. What's that like for you?"

Mitsuha's eyes light up with enthusiasm. "Oh, it's absolutely thrilling! Each piece tells a story, and being surrounded by so much creativity and expression is incredibly inspiring. I get to meet artists from all walks of life, and sometimes, I even get a behind-the-scenes look at their creative process. It's like being in a constant state of learning and discovery."

She then leans in a bit closer, her voice taking on a tone of excitement. "And you know, it's not just about the art on display. It's about the people who come to see it. Everyone interprets art differently, and hearing their perspectives can be just as enlightening as the art itself."

As the conversation flows, you notice that Mitsuha is genuinely passionate about her

I select option 2. TURN: 4 


Eager to dive deeper into her experiences, you ask Mitsuha about the most memorable piece of art she's encountered in her work. "With all the art you've seen, is there one piece that stands out to you the most?"

Mitsuha pauses for a moment, her gaze drifting off as if sifting through a mental gallery of countless artworks. Then, her face softens with a fond recollection. "Yes, there was one. It wasn't the most extravagant or famous piece, but it had a profound impact on me. It was a small, intricate painting by a relatively unknown artist. The piece depicted a quiet, serene landscape that seemed almost ethereal. But what struck me the most was the emotion it evoked—this sense of peace and solitude, but also a deep longing. It reminded me that art has the power to touch the soul in unexpected ways."

She looks back at you, her expression earnest. "It's moments like that which remind me why I love what I do. Art isn't just about aesthetics; it's about the e

I select option 1. TURN: 5 


Moved by her story, you share your own thoughts on how it resonated with your views on art. "Hearing about that painting and what it meant to you really strikes a chord with me. It's fascinating how art can evoke such strong emotions and connect with us on a personal level. I've always believed that art has this unique power to express what words cannot, to bridge gaps between people, and to inspire change or contemplation."

Mitsuha listens intently, nodding in agreement. "Exactly! It's that connection, that universal language of emotion and expression, that makes art so invaluable. I'm glad you feel the same way. It's rare to meet someone who truly understands the impact art can have."

The conversation between you two flows effortlessly, and it's clear that your shared passion for art has brought you closer. As the evening winds down, you both have learned much about each other's dreams, aspirations, and the profound respect you share for the world of a